In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re

def lower_str(columns):
    newlist = list(map(lambda x: x.lower(), columns))
    return newlist

def check_columns_received(df, bill_cols):
    twdb_col = lower_str(list(df.columns))
    col_miss = [i for i in bill_cols if i not in twdb_col]

    df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'], index=range(len(col_miss)))
    return df_col_missing

def replace_values(df, columns, value=""):
    """
    Está voltando para float
    """
    invalid_values = ['N/A', 'n/a',"0", 0, '-', '_', np.nan,'nan']
    #lista = []

    df.fillna(0, inplace=True)
    #df[column] = df[column].astype('int64')

    for column in columns:
        lista = []
        for index in df[column]:
            #print(f"{column} -> {index}")
            if index in invalid_values:
                lista.append(value)
            else:
                lista.append(index)
        df[column] = lista

    return df
# Refactorar esse codigo para receber todas as colunas num dic
# Sendo as keys=columns e values= picklist for each column
def check_date_columns(df, df_index, columns, format):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    df_dates = df[columns]
    df_dates['sites'] = df_dates[df_index]
    df_dates = df_dates.set_index('sites')

    df_de = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df_dates[df_index])
    if not df_duplicates.empty:
        df_dates.drop_duplicates(subset=[df_index], inplace=True)
    
    if format == 1:
        date_format = re.compile(r'\d{2}\-\d{2}\-\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                #print(type(df_dates.loc[de_site,de_column]))
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')
    else:
        date_format = re.compile(r'\d{2}\/\d{2}\/\d{4}')
        for de_site in df_dates[df_index]:
            for de_column in columns[1:]:
                #match = re.search(r'\d{2}\/\d{2}\/\d{4}', df_dates.loc[ta_site,ta_column])
                if date_format.match(df_dates.loc[de_site,de_column]) == None:
                    if df_dates.loc[de_site,df_index] not in df_de.index:
                        df_de.loc[de_site,df_index] = df_dates.loc[de_site,df_index]
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
                    else:
                        df_de.loc[de_site,de_column] = 'Incorret Format or Blank Value'
        df_de = df_de.dropna(how='all', axis=1).fillna('Ok')       
        if not df_de.empty:
            return df_de
        else: 
            print('No one columns with incorrect date format!')

def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')


def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    #Finding for ALL NEW SITES
    new_site = [str(i) for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    
    #list of new sites out of UIP sheet
    bts_out_uip = [str(i) for i in df_towerdb[df_towerdb[bts_col]=='Yes'][tw_index].sort_values() if i not in uip_list]
    bts_out_uip = pd.DataFrame(bts_out_uip, columns=['Bts_Sites_Out_UIP_File'])

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]

    return new_sites, bts_out_uip, df_site_bts[[tw_index, bts_col, bill_col]]
    """Restruturar o script em CZ, DE, PT"""

def check_bts(df_tw, bts_tw_columns, tw_index, df_msa, bts_msa_column, msa_index):
    #Nested Function to make conditional validations
    def cond_bts_check(bts_msa, tw_bts_sites):
        bts_out_tw=[]
        if sorted(bts_msa) != sorted(tw_bts_sites):
            for i in tw_bts_sites:
                if i not in bts_msa:
                    bts_out_tw.append(i)

        return bts_out_tw

    bts_msa = msa[msa[bts_msa_column]=='Yes']
    bts_msa = [str(i) for i in bts_msa[msa_index]]

    tw_bts_sites = df_tw[df_tw[bts_tw_columns]=='Yes']
    tw_bts_sites = [str(i) for i in tw_bts_sites[tw_index]]

    #return of datas
    bts_out_tw = cond_bts_check(bts_msa, tw_bts_sites)
    df = pd.DataFrame(bts_out_tw, columns=['New Sites'])
    return df

def check_wip(df_tw,tw_index, wip_tw, tw_bts, df_msa, msa_index, wip_msa_col):

    #Nested Function to make conditional validations
    def cond_wip_check(wip_msa, tw_wip_sites):
        count_wip = 0
        wip_out_tw=[]
        if sorted(wip_msa) != sorted(tw_wip_sites):
            for i in tw_wip_sites:
                if i not in wip_msa:
                    count_wip += 1
                    wip_out_tw.append(i)

        return wip_out_tw

    wip_msa = df_msa[df_msa[wip_msa_col]=='Yes']
    wip_msa = [str(i) for i in wip_msa[msa_index]]

    tw_wip_sites = df_tw[df_tw[wip_tw]=='Yes']
    tw_wip_sites = [str(i) for i in tw_wip_sites[tw_index]]

    tw_wip_site_bts_flagged = df_tw[(df_tw[wip_tw]=='Yes')&(df_tw[tw_bts]=='Yes')]
    tw_wip_site_bts_flagged = tw_wip_site_bts_flagged[[tw_index, wip_tw, tw_bts]]

    wip_out_tw_list = cond_wip_check(wip_msa, tw_wip_sites)
    return wip_out_tw_list, tw_wip_site_bts_flagged
    """Reestrurar os script em PT, DE, CZ"""
    # Falta os outros países

def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]
    """Ajustar para CZ, DE, PT"""
    
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')


def check_tw_doer_planned(df_tw, tw_index, doer_col, status_col):
    """Only GR until now"""
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[doer_col].empty:
        filtered = df_tw[(df_tw[status_col]=='Planned')&(not df_tw[doer_col].astype('datetime64[ns]').empty)]
        return filtered[[tw_index, status_col, doer_col]]  
    else:
        print('Nothing wrong in services sites!')
                                                              
def check_mom_bts(df_tw, tw_index, tw_col, df_msa,msa_index, msa_col):

    #c = country   
    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[tw_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = tw_bts[tw_bts[tw_index].isin(out_tower_bts)]
    return filtered[[tw_index, tw_col]]         

def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[df[start_date] > df[end_date]]
        return filtered[[tw_index, start_date,end_date]]


def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    return df_commercial_diffs


Adding new columns for UIP



In [ ]:
# Lendo o Ficheiro de input
path_uip=''
sheet = 'SiteLevel'
site_level = pd.read_excel(path_uip,sheet, header=[0,1,2])
head_1 = pd.MultiIndex.from_product([[''],['Numeric (in months)'],['Delay in Site Modification Projects', 'Delay in BTS Projects']])
df_1 = pd.DataFrame(columns=head_1)

head_2 = pd.MultiIndex.from_product([[''],['Numeric (in €)'],['Excess of Upgrade Capital Expenditure over Threshold']])
df_2 = pd.DataFrame(columns=head_2)

site_level = pd.concat([site_level, df_1, df_2], axis=1)
site_level.head(1)

Towerdb File

In [26]:
""" Need Unlock spreadsheet before"""
bill_cols = ['Site ID ','Strategic','Critical', 'Site Type (See structure Type)','Core Site Type','Transmission System',\
             'Macro Site - Transmission Hub Site','Macro Site - Transmission Hub Site with/without Shelters','Transmission Sites', 'Room Configuration',\
             'Power Supply','Air Conditioning','Strategic_Site_Bucket','CriticalSite_Beyond_10','Sites_As_Metered_Estimated','Meter_Sharing_Site','Bts_Sites',\
             'Phase1_Site_Or_Consent_Site','WIP_Sites','Active_Sharing_Arrangement','Subsequent_Sharing_Arrangement','First_Active_Sharing_Deployment_Type',\
             'Billing Trigger Date','Date of Equipment Removal']

path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet='TowerDB_Input_Ireland_20210630'
skiprows=0
skipcols=0
"""A coluna Infrastructure Ready Date contem 0 e deve ser feito replace"""
towerdb = pd.read_excel(path, sheet_name = sheet).fillna('')
towerdb = towerdb.dropna(subset=['Site ID '], axis=0)

towerdb['Infrastructure Ready Date'] = towerdb['Infrastructure Ready Date'].replace([0, '#N/D', 'N/A'],'')
lista = []
for dt in towerdb['Infrastructure Ready Date']:
    if dt != '':
        new_dt = f"{dt:%d/%m/%Y}"
        lista.append(new_dt)
    else:
        lista.append(dt)
towerdb['Infrastructure Ready Date'] = lista

towerdb['Billing Trigger Date'] = towerdb['Billing Trigger Date'].replace([0, '#N/D', 'N/A'],'')
lista = []
for dt in towerdb['Billing Trigger Date']:
    if dt != '':
        new_dt = f"{dt:%d/%m/%Y}"
        lista.append(new_dt)
    else:
        lista.append(dt)
towerdb['Billing Trigger Date'] = lista

#Read CZ towerdb files
"""Merge to get DOER column"""
msa = pd.read_csv('/content/TowerDB_Ireland_20210731.csv', engine='python')
doer_col = msa[['Site ID ', 'Date of Equipment Removal']]
towerdb = towerdb.merge(doer_col, on='Site ID ')

lista = []
towerdb['Date of Equipment Removal'] = towerdb['Date of Equipment Removal'].replace([0, '#N/D', 'N/A', np.nan, 'nan'],'')
for dt in towerdb['Date of Equipment Removal']:
    if dt != '' and re.match(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}", dt)==None:
        new_dt = f"{dt:%d/%m/%Y}"
        lista.append(new_dt)
    else:
        lista.append(dt)
towerdb['Date of Equipment Removal'] = lista

towerdb.rename(columns={'Billing Trigger Date': 'Billing Trigger Date.1'}, inplace=True)
towerdb['Billing Trigger Date'] = towerdb['Infrastructure Ready Date'].values

towerdb["Energy Cost FY1819"] = towerdb["Energy Cost FY1819"].apply(lambda x: '€ {:0,.2f}'.format(x))
towerdb['First_Active_Sharing_Start_Date'] = towerdb['First_Active_Sharing_Start_Date'].replace('-', '')
towerdb['First_Active_Sharing_End_Date'] = towerdb['First_Active_Sharing_End_Date'].replace('-', '')

cols = list(msa.columns)
towerdb = towerdb[cols]

invalid_values = ['N/A', 'n/a',"0" ,0, '-', '_', np.nan,'nan', ' ']
for col in bill_cols:
    towerdb[col] = towerdb[col].replace(invalid_values, '')

towerdb.rename(columns={'Billing Trigger Date.1': 'Billing Trigger Date'}, inplace=True)

towerdb.head(3)

,Site ID,Population,County,Site Name,Address,Site Provider,Strategic,Critical,Altitude (ASL),Lat (WGS 84),Long (WGS 84),Site Class,Site Owner,Hub List,Site Type (See structure Type),Active Dishes,INTP,Core Site Type,Transmission System,Macro Site - Transmission Hub Site,Macro Site - Transmission Hub Site with/without Shelters,Transmission Sites,Public DAS Sites (Active/ Passive),Fibre/Microwave,Technology VOD,Structure Type,Structure Type (Standardised),Room Configuration,Power Supply,Air Conditioning,Tower Height,Site Area (sqm),Site Area (Available),Macro Region,MPRN,Site Decom,SAP Site Code,Energy Cost FY1819,Active / Not Active,New Build planned,Site Status,Planned Sublets,Infrastructure Ready Date,Strategic_Site_Bucket,CriticalSite_Beyond_10,Sites_As_Metered_Estimated,Meter_Sharing_Site,Bts_Sites,Phase1_Site_Or_Consent_Site,Transfer_Date_Of_Consent_Site,Billing Trigger Date,WIP_Sites,Decommisioned_Site,Active_Sharing_Arrangement,Subsequent_Sharing_Arrangement,First_Active_Sharing_Deployment_Type,First_Active_Sharing_Start_Date,First_Active_Sharing_End_Date,BTS Site Acceptance Date,Billing Trigger Date,Date of Equipment Removal
0,CE009,183,Clare,Dangan,"Dangan, Ballyvaughan, Co Care",Private Individual,No,No,47,53.1011,-9.12983,Rural,Vodafone,P4,GBT,2,Standard,Non Core,Macro,No,Non Transmission Hub Site,Non Transmission Site,0,Microwave,2G; 3G,Lattice,Lattice,Indoor,AC,No,18,400,80,Mid West,1.00097e+10,In Service,IECE009,"€ 1,229.31",Active,,In Service,,01/08/1998,Non Strategic,Non Critical,Estimated Model,No,No,MSA (Phase 1),,01/08/1998,No,No,No Active Sharing,No,No,,,,,
1,CE015,1229,Clare,Bunratty Castle Hotel,Bunratty Castle Hotel Bunratty Clare,Hotel Company,No,No,12,52.6962,-8.81526,Rural,Private Company,P4,RTT,1,Standard,Non Core,Macro,No,Non Transmission Hub Site,Non Transmission Site,0,Microwave,2G; 3G; 4G,Rooftop structure,Rooftop structure,KN type Outdoor,AC,No,13.4,,,Mid West,0,In Service,IECE015,"€ 1,853.73",Active,,In Service,,07/05/2007,Non Strategic,Non Critical,Estimated Model,No,No,MSA (Phase 1),,07/05/2007,No,No,No Active Sharing,No,No,,,,,
2,CE020,1143,Clare,Liscannor Bay Hotel,Liscannor Bay View Hotel Liscannor Clare,Hotel Company,No,No,11,52.9382,-9.38929,Rural,Private Company,P4,RTT,1,Standard,Non Core,Macro,No,Non Transmission Hub Site,Non Transmission Site,0,Microwave,2G; 3G,Rooftop structure,Rooftop structure,KN type Outdoor,AC,No,19.2,,,Mid West,1.0019e+10,In Service,IECE020,€ 995.32,Active,,In Service,,21/05/2002,Non Strategic,Non Critical,Estimated Model,No,No,MSA (Phase 1),,21/05/2002,No,No,No Active Sharing,No,No,,,,,


In [ ]:
towerdb['Site ID '].value_counts()

In [10]:
towerdb['First_Active_Sharing_Start_Date'].value_counts()

-    1285
Name: First_Active_Sharing_End_Date, dtype: int64

In [ ]:
def change_incorret(df, df_index, col_chg, incorrect_value, new_value):
    list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

def change_incorret_list(df,list_sites, df_index, col_chg, new_value):
    #list_sites = list(df[df[col_chg]==incorrect_value][df_index])
    df.loc[df[df_index].isin(list_sites), col_chg] = new_value
    print(df.loc[df[df_index].isin(list_sites)][[df_index, col_chg]])

lista = ['CK166', 'CK173', 'CKMTG','DN370','DNTS1','DNVIC','GY135','KK074','RN053','RN058','SO071','WD067']
change_incorret_list(towerdb, lista, 'Site ID ', "Site Status", 'To be dismantled')

b_sites = ['CK094','DX264','GY025','DN947']
change_incorret_list(towerdb, b_sites,'Site ID ',"Bts_Sites", 'Yes')


In [ ]:
"""Precisa dos Critical sites da UIS"""
c_sites = critical['UIS Sites(critical beyond 10%) different in TowerDB'].to_list()
change_incorret_list(towerdb, c_sites,'Site ID ','CriticalSite_Beyond_10', 'Beyond 10%')

In [ ]:
towerdb.to_csv('/content/TowerDB_Ireland_20210831.csv', index=False)

In [18]:
"""Check columns received"""
def check_columns_received(df, bill_cols):
    twdb_col = list(df.columns)
    col_miss = [i for i in bill_cols if i not in twdb_col]

    if col_miss:
        df_col_missing = pd.DataFrame(col_miss, columns=['Column(s) Missing'])
        return df_col_missing
    else:
        print('\nNo missing columns.')

df_col_missing = check_columns_received(towerdb, cols)
df_col_missing
#Tem error 3 columns faltando

,Column(s) Missing
0,Billing Trigger Date.1


In [19]:
towerdb['Billing Trigger Date'].value_counts()

ValueError: ignored

In [ ]:
def check_date_columns(df, df_index,status_col,columns):
    """
    Paramns \n
        Dates needs to be in string format not in datetime, otherwise raise an error.\n
        Convert entire column to string before.
    """
    from collections import defaultdict

    df_dates = df[columns].fillna('')
    df_dates = df_dates.to_dict(orient='list')
    #df_dates['sites'] = df_dates[df_index]
    #df_dates = df_dates.set_index('sites')
    new_dic = defaultdict(list)

    date_format = re.compile(r"[0-9]{1,2}[\_|\-|\/][0-9]{1,2}[\_|\-|\/][0-9]{1,4}")

    for column in set(df_dates.keys()):
        for value in df_dates[column]:
            if date_format.match(value) == None:
                if value in (None, '', 'nan'):
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorret picklist value: {value}')
            else:
                new_dic[column].append('Ok')

    df_errors = pd.DataFrame(new_dic)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.replace('Ok', np.nan)   
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()
    
    df = df[[df_index, status_col]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[status_col]+ df_errors.columns[:-1].tolist()]
    df_errors = df_errors.reset_index()
    return df_errors


towerdb = towerdb.T.drop_duplicates().T
actives_1 = towerdb[towerdb['Site Status']=='In Service']
no_actives_1 = towerdb[~(towerdb['Site Status']=='In Service')]

#Checking columns for errors
actives_dates_errors = check_date_columns(actives_1, 'Site ID ', 'Site Status', ['Billing Trigger Date']) 
# Actives sites with blank billing trigger date
no_actives_dates_errors = check_date_columns(no_actives_1, 'Site ID ', 'Site Status', ['Date of Equipment Removal']) 

print(actives_dates_errors)
print('\n')
print(no_actives_dates_errors)

In [40]:
def check_picklist_v1(df,df_index,df_status, df_cols, picklist_dict):
    from collections import defaultdict

    df_picklist = df[df_cols].fillna('')
    df_picklist = df_picklist.to_dict(orient='list')

    new_dic = defaultdict(list)
    """for id in df_picklist[df_index]:
        if id not in new_dic.keys():
            new_dic[df_index].append(id)""" 
            
    for column in set(picklist_dict.keys()):
        for value in df_picklist[column]:
            if value not in set(picklist_dict[column]): 
                if pd.isnull(value) or pd.isna(value) or value=='' or value=='nan':
                    new_dic[column].append('Blank Value')
                else:
                    new_dic[column].append(f'Incorrect value: {value}')
            else:
                new_dic[column].append('Ok!')

    df_errors = pd.DataFrame(new_dic)
    df_errors = df_errors.replace('Ok!', np.nan)
    df_errors[df_index] = [i for i in df[df_index]]
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors.dropna(how='all', axis=0)
    df_errors = df_errors.dropna(how='all', axis=1)
    df_errors = df_errors.reset_index()

    df = df[[df_index, df_status]]
    df_errors = pd.merge(df_errors,df, how='left', on=[df_index])
    df_errors = df_errors.set_index(df_index)
    df_errors = df_errors[[df_status]+ df_errors.columns[:-1].tolist()]
    #df_errors = df_errors.reset_index()
    return df_errors

picklist = {
    "Strategic": ['Yes', 'No'],
    'Critical': ['Yes', 'No'],
    'Site Type (See structure Type)': ["DAS passive","GBT","RTT"],
    'INTP ': ["Standard","Core","Transmission"],
    'Core Site Type': ['Non Core', 'Case A', 'Case B'],
    'Transmission System': ["Macro","Public DAS","Long-term Mobile","Transmission"],
    'Macro Site - Transmission Hub Site': ['Yes', 'No'],
    'Macro Site - Transmission Hub Site with/without Shelters': ["With shelters","Without shelters","Non Transmission Hub Site"],
    'Room Configuration': ['Indoor','Outdoor', "KN type Outdoor","RBS type Outdoor"],
    'Power Supply': ['AC','DC', 'AC & DC','No Power'],
    'Air Conditioning': ['No','Yes; Indoor Air Conditioning','Yes; Indoor Free Air cooling / Free cooling units'],
    'Site Status': ['In Service','Ordered','Pipeline','Planned','To be dismantled'],
    'Strategic_Site_Bucket': ['Yes - 0-5%','Yes - 5-10%','Non Strategic'],
    'CriticalSite_Beyond_10': ['Beyond 10%','Within 10%','Non Critical'],
    'Sites_As_Metered_Estimated': ['Estimated Model','Metered Model'],
    'Meter_Sharing_Site': ['Yes', 'No'],
    "Bts_Sites": ['Yes', 'No'],
    'Phase1_Site_Or_Consent_Site': ['MSA (Phase 1)','PMA (Phase 2)'],
    'WIP_Sites': ['Yes', 'No'],
    'Active_Sharing_Arrangement': ['MORAN (On VF equipment)','MORAN (On non-VF equipment)','MOCN with Spectrum Pooling','Partial Active-Passive','No Active Sharing'],
    'Subsequent_Sharing_Arrangement': ['Yes', 'No']

}
"""Picklist check"""
actives = towerdb[towerdb['Site Status']=='In Service']
col_pick =['Site ID ',"Strategic",'Critical','Site Type (See structure Type)','INTP ','Core Site Type','Transmission System',\
    'Macro Site - Transmission Hub Site','Macro Site - Transmission Hub Site with/without Shelters',\
    'Room Configuration','Power Supply','Air Conditioning','Site Status','Strategic_Site_Bucket','CriticalSite_Beyond_10',\
    'Sites_As_Metered_Estimated','Meter_Sharing_Site',"Bts_Sites",'Phase1_Site_Or_Consent_Site','WIP_Sites','Active_Sharing_Arrangement',\
    'Subsequent_Sharing_Arrangement']
"""Picklist check"""
actives = towerdb[towerdb['Site Status']=='In Service']
df_picklist = check_picklist_v1(actives, 'Site ID ','Site Status', col_pick, picklist)
df_picklist

,Site Status,Subsequent_Sharing_Arrangement
Site ID,,
WH032,In Service,Blank Value
DN304,In Service,Blank Value
DN947,In Service,Blank Value
DX252,In Service,Blank Value


In [46]:
"""MoM check sites"""
def check_mom_bts(df_tw, tw_index,status_col, bts_col, df_msa, msa_index, msa_col):

    msa_bts = df_msa[df_msa[msa_col]=='Yes']
    msa_bts_sites = [i for i in msa_bts[msa_index]]

    tw_bts = df_tw[df_tw[bts_col]=='Yes']
    tw_bts_sites = [i for i in tw_bts[tw_index]]

    out_tower_bts = [i for i in msa_bts_sites if i not in tw_bts_sites]
    filtered = df_msa[df_msa[tw_index].isin(out_tower_bts)]
    filtered = filtered[[tw_index,status_col, bts_col]]
    actual = df_tw[df_tw[tw_index].isin(out_tower_bts)][[tw_index, bts_col, status_col]]
    merge = pd.merge(filtered, actual, how='inner', on=tw_index, suffixes=['_old', '_in_month'])

    return  merge


df_mom = check_mom_bts(towerdb, 'Site ID ','Site Status', "Bts_Sites", msa ,'Site ID ', "Bts_Sites")
df_mom

,Site ID,Site Status_old,Bts_Sites_old,Bts_Sites_in_month,Site Status_in_month
0,CK094,In Service,Yes,No,In Service
1,DN947,In Service,Yes,No,In Service
2,DX264,In Service,Yes,No,In Service
3,GY025,In Service,Yes,No,In Service


In [ ]:
path_uip = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', \
             'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2).fillna('')
uip.columns = uip_names

msa_sites = [i for i in msa['Site ID ']]
tw_sites = [i for i in towerdb ['Site ID ']]
uip_sites = [i for i in uip['Site_ID']]


In [ ]:
uip['Commercials for sites beyond 10% cap of critical sites (Annual)'].value_counts()

          1125
1000.0      95
900.0       31
750.0        2
Name: Commercials for sites beyond 10% cap of critical sites (Annual), dtype: int64

In [ ]:
""" BTS sites"""
def check_new_sites(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list):
    
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    
    filtered = df_towerdb[[tw_index, status_col]]

    #Finding for ALL NEW SITES
    new_site = [i for i in towerdb_list if i not in msa_list]
    new_sites = pd.DataFrame(new_site, columns=['New_Sites'])
    new_sites = pd.merge(new_sites, filtered, how='left', left_on=['New_Sites'], right_on=tw_index)
    new_sites = new_sites[['New_Sites', status_col]]

    
    #list of new sites out of UIP sheet
    bts_out_uis = [i for i in df_towerdb[(df_towerdb[bts_col]=='Yes')&(df_towerdb[status_col]=='In Service')][tw_index] if i not in uip_list]
    bts_out_uis = pd.DataFrame(bts_out_uis, columns=['Bts_Sites_Out_UIS_File'])
    bts_out_uis = pd.merge(bts_out_uis, filtered, how='left', left_on=['Bts_Sites_Out_UIS_File'], right_on=tw_index)
    bts_out_uis = bts_out_uis[['Bts_Sites_Out_UIS_File', status_col]]

    #create a copy to make index modifications
    df = df_towerdb.copy()

    #New columns with Site Codes
    df['Sites'] = df[tw_index]
    #defining site code to index
    df.set_index('Sites', inplace=True)

    #filtering by new sites
    df = df[df[tw_index].isin(new_site)]

    # Save information os sites with demerged date more than current date
    print(df[bill_col])
    df[bill_col] = df[bill_col].astype('datetime64[s]')
    df_site_bts = df[(df[bts_col]=='Yes') | (df[bill_col] > current_date)]
    
    #if not (new_sites.empty or bts_out_uis.empty or df_site_bts.empty):
    return new_sites, bts_out_uis, df_site_bts[[tw_index,status_col, bts_col, bill_col]]

msa_sites = [i for i in msa['Site ID ']]
tw_sites = [i for i in towerdb ['Site ID ']]
uip_sites = [i for i in uip['Site_ID']]
#(df_towerdb, tw_index, bts_col, bill_col, status_col, msa_list, towerdb_list, uip_list)
new_sites, bts_out_uis, df_site_bts = check_new_sites(towerdb, 'Site ID ', "Bts_Sites", \
                                                      'Billing Trigger Date','Site Status', msa_sites, tw_sites, uip_sites)
#no errors

Series([], Name: Billing Trigger Date, dtype: object)


In [ ]:
"""BTS Wip"""
wip_out_tw_list, df_wip_and_bts_flagged = check_wip(actives,'Site ID ','WIP_Sites', "Bts_Sites", msa, 'Site ID ', "Bts_Sites")
# No errors

In [ ]:
"""Decomissioned Sites"""
def check_decommissioned(df,df_index, decom_col, doer_col):
    #c = country.lower()
    filtered = df[(df[decom_col]=='Yes')&(df[doer_col]=="")]
    return filtered[[df_index, decom_col, doer_col]]

df_decom = check_decommissioned(actives,'Site ID ','Decommisioned_Site', 'Date of Equipment Removal')
df_decom
# No errors

In [ ]:
"""Check doer in service sites"""
def check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col):
    
    t = type_col.lower()
    # capture current date as string
    current_date = pd.to_datetime('now').date()
    # convert current to timestamp
    current_date = pd.to_datetime(current_date)
    if not df_tw[date_col].empty:
        if t == 'doer':
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]') < current_date)]
            return filtered[[tw_index, status_col, date_col]] 
        else:
            filtered = df_tw[(df_tw[status_col]==status)&(df_tw[date_col].astype('datetime64[ns]').empty)]
            return filtered[[tw_index, status_col, date_col]] 
    else:
        print('Nothing wrong in services sites!')
#check_tw_bill_doer(df_tw, tw_index, date_col, status_col, status, type_col)       
df_doer = check_tw_bill_doer(actives, 'Site ID ', 'Date of Equipment Removal','Site Status','In Service', 'doer')
df_doer
# No errors

,Site ID,Site Status,Date of Equipment Removal


In [ ]:
towerdb['CriticalSite_Beyond_10'].value_counts()

Non Critical    1046
Within 10%       238
                   1
Name: CriticalSite_Beyond_10, dtype: int64

In [ ]:
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
    #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = []
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS In Month not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS In Month not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS In Month not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS In Month!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS In Month!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS In Month!', status_tw_col]]
    #check for decomissioned site not in uip files

    tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
    decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
    decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                    right_on=tw_index)
    decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]

    #Check BTS sites
    bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
    uip_bts = [i for i in df_uip[df_uip['BTS site applicable charge (Annual)']!='']['Site_ID']]

    bts_sites_out_uip = []
    #if not set(bts_sites).intersection(uip_sites):
    bts_sites_out_uip = [i for i in uip_bts if i not in bts_sites]
    bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['UIS BTS not in TowerDB(BTS)'])
    bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='UIS BTS not in TowerDB(BTS)',\
                                    right_on=tw_index)
    bts_sites_out_uip = bts_sites_out_uip[['UIS BTS not in TowerDB(BTS)', status_tw_col]]

    #  Check for UIP critical sites 
    uip_critical = [i for i in df_uip[df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='']['Site_ID']]
    bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
    print(len(uip_critical))
    critical = []
    if len(uip_critical) > 0:
        #if set(uip_critical).intersection(bts_tw_critical):
        critical = [i for i in uip_critical if i not in bts_tw_critical]
        critical = pd.DataFrame(critical, columns=['UIS Sites(critical beyond 10%) different in TowerDB'])
        critical = pd.merge(critical, filtered, how='left', left_on='UIS Sites(critical beyond 10%) different in TowerDB',\
                                    right_on=tw_index)
        critical = critical[['UIS Sites(critical beyond 10%) different in TowerDB', status_tw_col]]
    #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
    return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    


"""Check UIP sites matches with Towerdb sites"""
uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical = check_uip_tw(towerdb,'Site ID ', 'Site Status', \
                                                              'Decommisioned_Site', "Bts_Sites",'CriticalSite_Beyond_10', \
                                                              uip, uip_sites)

128


In [ ]:
c_sites = critical['UIS Sites(critical beyond 10%) different in TowerDB'].to_list()


In [ ]:
bts_sites_out_uip, critical

,Sites with critical level beyond 10% in out UIS File,Site Status
0,CN049,In Service
1,LSRGB,In Service
2,MN008,In Service
3,TY002,In Service
4,DN295,In Service
...,...,...
123,CE007,In Service
124,DN565,In Service
125,MH042,In Service
126,RNSCE,In Service


In [ ]:
uis_sites_not_in_towerdb

,UIS In Month not active in TowerDB!,Site Status
0,DN354,Dismantled
1,RN038,Dismantled


In [ ]:
in_service_not_in_uis

,TowerDB Sites out of UIS In Month!,Site Status
0,DN433,In Service
1,DN822,In Service
2,DNTWR,In Service
3,DN320,In Service
4,DNCF1,In Service
5,DNNGE,In Service
6,WH032,In Service
7,DN304,In Service


In [ ]:
"""Check UIP sites matches with Towerdb sites"""
def check_uip_tw(df_tw,tw_index, status_tw_col, decom_col, tw_bts_col, tw_critical_col, df_uip, uip_sites):

    filtered = df_tw[[tw_index, status_tw_col]]
        #tw_active = df_tw[df_tw['Site Status']=='In Service']
    count_tw_sites = [i for i in df_tw[df_tw[status_tw_col] =='In Service'][tw_index]]

    # check number of sites that are in uip file and doesn't have in df_tw
    uis_sites_not_in_towerdb = []
    #if not set(count_tw_sites).intersection(uip_sites):
    uis_sites_not_in_towerdb = [i for i in uip_sites if i not in count_tw_sites]
    if uis_sites_not_in_towerdb:
        uis_sites_not_in_towerdb = pd.DataFrame(uis_sites_not_in_towerdb,columns=['UIS Sites True Up not active in TowerDB!'])
        uis_sites_not_in_towerdb = pd.merge(uis_sites_not_in_towerdb, filtered, how='left', left_on='UIS Sites True Up not active in TowerDB!',\
                                        right_on=tw_index)
        uis_sites_not_in_towerdb = uis_sites_not_in_towerdb[['UIS Sites True Up not active in TowerDB!', status_tw_col]]
    
    in_service_not_in_uis = [i for i in count_tw_sites if i not in uip_sites]
    in_service_not_in_uis = pd.DataFrame(in_service_not_in_uis,columns=['TowerDB Sites out of UIS True Up Sites!'])
    in_service_not_in_uis = pd.merge(in_service_not_in_uis, filtered, how='left', left_on='TowerDB Sites out of UIS True Up Sites!',\
                                    right_on=tw_index)
    in_service_not_in_uis = in_service_not_in_uis[['TowerDB Sites out of UIS True Up Sites!', status_tw_col]]
    #check for decomissioned site not in uip files

    if decom_col != "":
        tw_decomiss = [i for i in df_tw[df_tw[decom_col]=='Yes'][tw_index] if i in uip_sites]
        decomiss_sites_in_uip = pd.DataFrame(tw_decomiss, columns=['Decomissioned Site in UIS File'])
        decomiss_sites_in_uip = pd.merge(decomiss_sites_in_uip, filtered, how='left', left_on='Decomissioned Site in UIS File',\
                                        right_on=tw_index)
        decomiss_sites_in_uip = decomiss_sites_in_uip[['Decomissioned Site in UIS File', status_tw_col]]
    
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        
        critical = []
        if len(bts_tw_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
                #print(critical)
        if critical:
            critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
            critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
            critical = critical[['Sites with critical level beyond 10% in out UIPS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or decomiss_sites_in_uip.empty or bts_sites_out_uip.empty or critical.empty):
        return uis_sites_not_in_towerdb, in_service_not_in_uis, decomiss_sites_in_uip, bts_sites_out_uip, critical
    else:
        #Check BTS sites
        bts_sites = [i for i in df_tw[df_tw[tw_bts_col]=='Yes'][tw_index]]
        bts_sites_out_uip = []
        if not set(bts_sites).intersection(uip_sites):
            bts_sites_out_uip = [i for i in bts_sites if i not in uip_sites]
        bts_sites_out_uip = pd.DataFrame(bts_sites_out_uip, columns=['BTS Site not in UIS File'])
        bts_sites_out_uip = pd.merge(bts_sites_out_uip, filtered, how='left', left_on='BTS Site not in UIS File',\
                                        right_on=tw_index)
        bts_sites_out_uip = bts_sites_out_uip[['BTS Site not in UIS File', status_tw_col]]

        #  Check for UIP critical sites 
        uip_critical = [i for i in df_uip[(df_uip['Commercials for sites beyond 10% cap of critical sites (Annual)']!='')&\
                                    df_uip['BTS site applicable charge (Annual)']!=""]['Site_ID']]
        bts_tw_critical = df_tw[df_tw[tw_critical_col]=='Beyond 10%'][tw_index]
        critical = []
        if len(uip_critical) > 0:
            if set(uip_critical).intersection(bts_tw_critical):
                critical = [i for i in bts_tw_critical if i not in uip_critical]
        critical = pd.DataFrame(critical, columns=['Sites with critical level beyond 10% in out UIS File'])
        critical = pd.merge(critical, filtered, how='left', left_on='Sites with critical level beyond 10% in out UIS File',\
                                        right_on=tw_index)
        critical = critical[['Sites with critical level beyond 10% in out UIS File', status_tw_col]]
        #if not (in_service_uip_sites.empty or bts_sites_out_uip.empty or critical.empty):
        return in_service_uip_sites, bts_sites_out_uip, critical


path_uiptrue = '/content/UserInput_Ireland_20210630 June delta.xlsx'
uip_names = ['Site_ID', 'BTS site applicable charge (Annual)', 'Commercials for sites beyond 10% cap of critical sites (Annual)']
uip_true = pd.read_excel(path_uip ,sheet_name='SiteLevel',usecols=[0,1,2],skiprows=2)
uip_true.columns = uip_names
uip_sites_true = [i for i in uip_true['Site_ID']]

uis_true_not_in_towerdb,in_service_uis_true, decomiss_in_uis, bts_sites_out_uis_true, critical_true = check_uip_tw(towerdb,'Site ID ', 'Site Status', \
                                                                'Decommisioned_Site', "Bts_Sites",'CriticalSite_Beyond_10', uip, uip_sites)
#No errors


In [ ]:
uis_true_not_in_towerdb

,UIS Sites True Up not active in TowerDB!,Site Status
0,DN354,Dismantled
1,RN038,Dismantled


In [ ]:
in_service_uis_true

,TowerDB Sites out of UIS True Up Sites!,Site Status
0,DN433,In Service
1,DN822,In Service
2,DNTWR,In Service
3,DN320,In Service
4,DNCF1,In Service
5,DNNGE,In Service
6,WH032,In Service
7,DN304,In Service


In [ ]:
def check_diffs_v2(path_current, path_last, sheet='Commercial'):
    
    def lower_str(columns):
        newlist = list(map(lambda x: x.lower(), columns))
        return newlist

    def highlight_diff(data, color='yellow'):
        attr = 'background-color: {}'.format(color)
        other = data.xs('Current', axis='columns', level=-1)
        return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                            index=data.index, columns=data.columns)
    #type_file = type_file.lower()
    #if type_file =='excel':
    _actual = pd.read_excel(path_current,sheet_name=sheet).fillna('')
    _before = pd.read_excel(path_last,sheet_name=sheet).fillna('')

    df_all = pd.concat([_actual, _before],axis='columns', keys=['Current', 'Last'])
    df_final = df_all.swaplevel(axis='columns')[_actual.columns[1:]]

    #df_final.style.apply(highlight_diff, axis=None)
    if not df_final.empty:
        return df_final[(_actual != _before).any(1)].style.apply(highlight_diff, axis=None)
    else:
        print('\nNo differences Founded!\n')
path_before = '/content/UserInput_Ireland_20210731.xlsx'
path_curr = '/content/UserInput_Ireland_July 21_06.07.2021.xlsx'
df_comm = check_diffs_v2(path_curr, path_before)
df_comm
# No errors

In [ ]:
def check_commercial(path_current, path_before, col_replace, col_names, merge_cols, col_order):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan']
        #lista = []

        df.fillna("", inplace=True)
        #df[column] = df[column].astype('int64')

        for column in columns:
            lista = []
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista

        return df

    def check_uip_commercial_values(df_actual, df_before, merge_cols):
        df_atual = uip_comercial_actual
        df_ant = uip_comercial_before
        df_cross = pd.merge(df_actual, df_before, on=merge_cols, \
                            how='left', suffixes=('_actual', '_before'), indicator='Exist')
        df_cross['Exist'] = np.where(df_cross.Exist == 'both', 'Yes', 'No')
        df_cross['Equal Values'] = df_cross['Exist']
        
        return df_cross
    # Check for commercial Values into current UIP File and compare with UIP File before
    uip_comercial_actual = pd.read_excel(path_current,sheet_name='Commercial', names=col_names)
    #uip_comercial_actual = uip_comercial_actual[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_actual = replace_values(uip_comercial_actual, col_replace, value=0)

    uip_comercial_before = pd.read_excel(path_before,sheet_name='Commercial', names=col_names )
    #uip_comercial_before = uip_comercial_before[['Charge_Type', 'Data_Type', 'Input_Value']]
    uip_comercial_before = replace_values(uip_comercial_before, col_replace, value=0)

    df_commercial =  check_uip_commercial_values(uip_comercial_actual, uip_comercial_before, merge_cols)

    #df_commercial = df_commercial.reindex(columns=col_order)
    df_commercial_diffs = df_commercial[df_commercial['Equal Values']=='No']
    if not df_commercial_diffs.empty:
        return df_commercial_diffs
    else:
        print('\nNo Errors founded!')

names = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', 'Input_Value',\
         'Description/Instruction', 'Frequency of Update']
merge_cols = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2', 'Data_Type', \
              'Description/Instruction', 'Frequency of Update']
cols_ordered = ['Charge_Type','Sub_Charge_Type', 'Param1','Param2','Data_Type','Input_Value_actual',\
                'Input_Value_before','Equal Values','Description/Instruction', 'Frequency of Update']
# Check for commercial Values into current UIP File and compare with UIP File before

df_com = check_commercial(path_curr, path_before,['Input_Value'], names, merge_cols, cols_ordered)
df_com


No Errors founded!


Excel log

In [ ]:
df_tw = [bts_sites_out_uip, critical]
sheetnames_tw = ['UIS BTS out of TWDB(BTS)','UIS Critical outof Towerdb']

path_tw = '/content/towerdb_IE_errors.xlsx'
general_log_erros(df_tw, sheetnames_tw, path_tw)

TA Input File

In [ ]:
#Comparando os ficheiros
pathta='/content/TowerDB_Ireland_20210531 1.xlsx'
pathold = '/content/TA_Input_Ireland_20210630.csv'
sheet='TA_Input_Ireland_20210531'
skiprows=0
skipcols=0
ta_save = '/content/TA_IE'
ta_bills = ['Site ID ','Customer Type ','Lease Start Date ', 'Lease End date ']
find_diffs_between_files(pathold, pathta, 'Site ID ', ta_bills,'',ta_save, 'mix','ta',sheet)


New Rows:     ['DL020 ', 'MNCVG']
Dropped Rows: []

Done.



In [ ]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
    #header=0, names = col_name,
    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df[col_name] 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    df = replace_values(df, cols_int, 0)
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    df = replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df
ta_col_ord = ["Site ID ","Site Name ","Address ","Site Provider","Site type",\
              "Tenant/Customer ","Customer Type "," Initial Rent  "," Current Rent  "," REC Uplift Charge ",\
              "Current Power Charge","Renegotiations ","Lease Start Date ","Lease End date ","Right to Renew ",\
              "End of Renewal Period ","Payment Terms ","Payment Frequency ","Rent Review ","Indexation Driver",\
              "Fixed Increase ","CPI ","Percentage","Increase frequency","Increase Date","VAT","Rate ","Termination date "]
interger = ['Increase frequency']
amount = []
ta_bill_cols = ["Site ID ","Tenant/Customer ","Customer Type ",\
                "Lease Start Date ","Lease End date "]
ta_dates = ["Lease Start Date ","Lease End date ", "Increase Date"]
              
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet='TA_Input_Ireland_20210630'
skiprows=0
skipcols=0
#(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               #cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y")
ta = read_files(path, sheet,ta_col_ord, 0, 0, "Site ID ", ta_dates, interger,amount, ta_bill_cols)
ta.head(3)

In [ ]:
ta.head(3)

,Site ID,Site Name,Address,Site Provider,Site type,Tenant/Customer,Customer Type,Initial Rent,Current Rent,REC Uplift Charge,Current Power Charge,Renegotiations,Lease Start Date,Lease End date,Right to Renew,End of Renewal Period,Payment Terms,Payment Frequency,Rent Review,Indexation Driver,Fixed Increase,CPI,Percentage,Increase frequency,Increase Date,VAT,Rate,Termination date
0,CE007,Lack West,Lack West Kilmihil Clare,Vodafone,GBT,Three Ireland Services (Hutchinston) Ltd,MNO,,14417.338624,,Vfi supply,,06/02/2006,05/02/2011,No,,Quarterly in Advance,4,CPI Applied from 01 October annually,Annually Tenant Fees,CPI/4% (the greater of),-,1,12,01/10/2019,Yes,23%,
1,CE007,Lack West,Lack West Kilmihil Clare,Vodafone,GBT,Tetra Ireland Communications Ltd,OTMO,9500,10142.700000,,531,,01/10/2011,30/09/2021,No,,Half Yearly,2,Annually,Annually Tenant Fees,CPI,-,1,12,01/10/2019,Yes,23%,
2,CE007,Lack West,Lack West Kilmihil Clare,Vodafone,GBT,Imagine,OTMO,12000,12000.000000,,Vfi supply,,25/07/2019,24/07/2024,No,,Quarterly In Advance,4,-,-,-,-,-,0,-,Yes,23%,


In [ ]:
ta['Percentage']

In [ ]:
ta.to_csv('/content/TA_Input_Ireland_20210831.csv', index=False)

In [ ]:
cols_date = ['Site ID ', 'Lease Start Date ', 'Lease End date ']
df_dates_erros = check_date_columns(ta, 'Site ID ', cols_date, 2)
# No errors

In [ ]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[pd.to_datetime(df[start_date]) > pd.to_datetime(df[end_date])]
        return filtered[[tw_index, start_date, end_date]]

df_ta_dates = check_lc_ta_dates(ta,"Site ID ", "Lease Start Date ","Lease End date ")
df_ta_dates

,Site ID,Lease Start Date,Lease End date


In [ ]:
"""Picklist Values"""
picklist_ta = {
    'Customer Type ': ['MNO','OTMO']
}
ta['Customer Type '].value_counts()

MNO     494
OTMO    229
Name: Customer Type , dtype: int64

LC Input File

In [ ]:
#Comparando os ficheiros
pathlc='/content/TowerDB_Ireland_20210531 1.xlsx'
pathold_lc = '/content/LC_Input_Ireland_20210630.csv'
sheet_lc='LC_Input_Ireland_20210531'
skiprows=0
skipcols=0
lc_save = '/content/LC_IE'
lc_bills = ['Site ID ', ' Total Rent ', 'Lease Start Date ', 'Lease End date ']
find_diffs_between_files(pathold_lc, pathlc, 'Site ID ', lc_bills,'',lc_save, 'mix','ta',sheet_lc)

LC Validations

In [ ]:
#Read TA Input File
def read_files(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y"):
    
    def replace_values(df, columns, value=""):
        """
        Está voltando para float
        """
        invalid_values = ['N/A', 'n/a',"0", '-', '_', np.nan,'nan', ' ', 'not available yet']

        for column in columns:
            lista = []
            df[column] = df[column].fillna(0)
            for index in df[column]:
                #print(f"{column} -> {index}")
                if index in invalid_values:
                    lista.append(value)
                else:
                    lista.append(index)
            df[column] = lista
        return df
    def date_parser(df, columns,format, type_date):
        for column in columns:
            if format == 'mix':
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) \
                            and not isinstance(date_obj, str) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
            else:
                df[column] = [date_obj.strftime(type_date) if not pd.isnull(date_obj) else '' for date_obj in df[column]]
                df[column] = df[column].astype(str)
    #header=0, names = col_name,
    df = pd.read_excel(path, sheet_name = sheetname, engine='openpyxl', skiprows = n_skiprows)
    df = df.iloc[:,n_skip_columns:]
    df = df[col_name] 
    df = df.dropna(subset=[site_index], axis=0)
    df = replace_values(df, cols_amount, 0)
    for col in cols_amount:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int).apply(lambda x: f'{x:,}')
    df = replace_values(df, cols_int, 0)
    for col in cols_int:
        df[col] = df[col].fillna(0)
        df[col] = df[col].astype(int)
    df = df.fillna('')
    df = replace_values(df, bill_cols)
    date_parser(df, cols_date, format, type_date)
    # define the entiry columns which doesn't have NaN 
    return df
lc_ord = ["Site ID ","Site Name ","Address ","Site Provider","Contract Type","Vendor"," Current Rent  ",\
          " Sublet/Uplift "," Total Rent ","Misc Fees ","Sublet Clause ","Conditions ","Uplift for Sublet ",\
          "Lease Start Date ","Lease End date ","Right to Renew ","End of Renewal Period ","Payment Terms ",\
          "Payment Frequency","Rent Review ","Fixed Increase ","Indexation Driver","CPI/Increase Type",\
          "Percentage","Increase Frequency","Increase date","VAT Subject","VAT","Termination date ","Observations"]
interger = []#'Increase frequency'
amount = []#" Current Rent  "," Total Rent "
ta_bill_cols = ["Site ID ",\
                "Lease Start Date ","Lease End date "]
dates = ["Lease Start Date ","Lease End date ", "Increase date"]             
path='/content/TowerDB_Ireland_20210630 v1.xlsx'
sheet_lc='LC_Input_Ireland_20210630'
#(path, sheetname,col_name, n_skiprows, n_skip_columns, site_index, cols_date, cols_int,\
               #cols_amount, bill_cols, format='mix', type_date="%d/%m/%Y")
lc = read_files(path, sheet_lc,lc_ord, 0, 0, "Site ID ", dates, interger,amount, ta_bill_cols)
#lc["Uplift for Sublet "] = [re.sub('^€', '', str(x)) for x in lc["Uplift for Sublet "]]
#lc["Uplift for Sublet "] = lc["Uplift for Sublet "].replace('-', '')
lc["VAT"] = lc["VAT"].replace(0.23, '23%')
lc.loc[lc["Percentage"]==1, "Percentage"] = '100%'
lc.loc[lc["Lease Start Date "]=='NaT', "Lease Start Date "] = '100%'
#lc.loc[lc["Lease Start Date "]=='NaT', "Lease Start Date "] = ''
#lc.loc[lc["Lease End date "]=='NaT', "Lease End date "] = ''
lc.head()

,Site ID,Site Name,Address,Site Provider,Contract Type,Vendor,Current Rent,Sublet/Uplift,Total Rent,Misc Fees,Sublet Clause,Conditions,Uplift for Sublet,Lease Start Date,Lease End date,Right to Renew,End of Renewal Period,Payment Terms,Payment Frequency,Rent Review,Fixed Increase,Indexation Driver,CPI/Increase Type,Percentage,Increase Frequency,Increase date,VAT Subject,VAT,Termination date,Observations
0,CE030,Cloonnagarnaun,Cloonnagarnaun Creegh Cloghaun Beg Clare,Cignal,LEASE,Cignal Infrastructure Limited,10558.360000,0.0,10558.4,No,Yes,Uplift,3750,29/12/1998,28/12/2098,No,-,Quarterly in advance,4,Yes,No,Annual Lease Fees,CPI/18.5% (the greater of),100%,60,29/12/2023,Yes,23%,-,
1,CK013,Bishopstown ESB,Bishopstown 38Kv Stn Bishopstown Cork,ESB,LICENCE,Esb Telecoms Ltd,5842.808501,0.0,5842.81,No,No,-,-,01/01/2011,31/12/2025,No,-,Half yearly in advance by gale date,2,Yes,No,Annual Lease Fees,CPI/0.5% (the lesser of),100%,12,01/01/2020,Yes,23%,-,
2,CK071,Carrig,"Knockgarriff, Carrig, Ballinascarthy, Co Cork ...",Private Individual,LEASE,Shay Nyhan,8000.000000,0.0,8000,No,Yes,Uplift,2500,21/10/2018,20/10/2028,No,-,yearly in advance by gale date,1,Yes,No,Annual Lease Fees,CPI,100%,60,21/10/2023,No,No VAT,-,
3,CK079,Awnaskirtawn,"Awnaskirtawn,",Cignal,LEASE,Cignal Infrastructure Limited,10020.000000,0.0,10020,No,Yes,Uplift,3750,17/12/1999,16/12/2099,No,-,Quarterly in advance,4,Yes,No,Annual Lease Fees,CPI/18.5% (the greater of),100%,60,17/12/2024,Yes,23%,-,
4,CK087,Curraghanaltig,Curraghanaltig,Cignal,LEASE,Cignal Infrastructure Limited,10017.000000,0.0,10017,No,Yes,Uplift,€3750,16/09/1999,15/09/2099,No,-,Quarterly in advance,4,Yes,No,Annual Lease Fees,CPI/18.5% (the greater of),100%,60,16/09/2024,Yes,23%,-,


In [ ]:
lc.loc[lc["Lease Start Date "]=='NaT', "Lease Start Date "]

Series([], Name: Lease Start Date , dtype: object)

In [ ]:
lc.to_csv('/content/LC_Input_Ireland_20210831.csv', index=False)

In [ ]:
def check_lc_ta_dates(df,tw_index, start_date,end_date):

        filtered = df[pd.to_datetime(df[start_date]) > pd.to_datetime(df[end_date])]
        return filtered[[tw_index, start_date, end_date]]

df_lc_dates = check_lc_ta_dates(lc,"Site ID ", "Lease Start Date ","Lease End date ")
df_lc_dates

,Site ID,Lease Start Date,Lease End date


In [ ]:
def check_amounts(df_check, df_index, columns, pattern=','):
    """
    Paramns:
    pattern: general (. to decimal), other (, to decimal)
    """

    df = df_check[columns]
    df['sites'] = df_check[df_index]
    df = df.set_index('sites')

    df_new = pd.DataFrame(columns=columns)
    
    df_duplicates = count_duplicates(df[df_index])
    if not df_duplicates.empty:
        df.drop_duplicates(subset=[df_index], inplace=True)

    for site in df[df_index]:
        for column in columns[1:]:
            print(df.loc[site,column])
            if not str(df.loc[site,column]).__contains__(pattern):
                #print(df.loc[site,column])
                if df.loc[site,df_index] not in df_new.index:
                    df_new.loc[site,df_index] = df.loc[site,df_index]
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'
                else:
                    df_new.loc[site,column] = 'Incorret Format to separate decimal values'

    df_new = df_new.dropna(how='all', axis=1).fillna('Ok')       
    if not df_new.empty:
        return df_new
    else: 
        print('No one columns with incorrect Amount format!')

"""Check Amount"""
lc_col_amounts = [' Total Rent ']
df_lc_amounts = check_amounts(lc, "Site ID ", lc_col_amounts, 2)
# No errors

In [ ]:
df_lc = [df_lc_dates_erros]
sheetnames_lc = ["Sites In Service with incorrect date"]

path_lc = '/content/lc_ie_errors.xlsx'
general_log_erros(df_lc, sheetnames_lc, path_lc)

/usr/local/lib/python3.7/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
